# Point Cloud Visualization

In [324]:
from platform import python_version
print(python_version())

3.10.0


In [325]:
import numpy as np
import json
import open3d as o3d
import matplotlib.pyplot as plt

### Format: pcd | Package: Open3D

#### Получениеф файла pcd

In [326]:
geometries = []
cloud = o3d.io.read_point_cloud("D:/university/6semestr/AI-diploma-niias/stasik/src/cloud_2_0001.pcd")
geometries.append(cloud)
print(np.asarray(cloud.points))
print(np.asarray(cloud.colors))
print(np.asarray(cloud.normals))
print(cloud)
print('Shape of points', np.asarray(cloud.points).shape)
print('Shape of colors', np.asarray(cloud.colors).shape) # no color data

[[ 1.836      -0.34200001  3.7860003 ]
 [ 1.839      -0.34200001  3.7450001 ]
 [ 1.842      -0.34200001  3.7320001 ]
 ...
 [ 1.8660001   1.077       3.3440001 ]
 [ 1.869       1.077       3.3440001 ]
 [ 1.872       1.077       3.3330002 ]]
[[0.66666667 0.         1.        ]
 [0.63921569 0.         1.        ]
 [0.62745098 0.         1.        ]
 ...
 [0.35294118 0.         1.        ]
 [0.35294118 0.         1.        ]
 [0.35294118 0.         1.        ]]
[]
PointCloud with 526833 points.
Shape of points (526833, 3)
Shape of colors (526833, 3)


#### Извлечение данных json и построение кубойдов

In [327]:
def create_cuboid(cub):
    position = cub['position']
    rotation = cub['rotation']
    dimensions = cub['dimensions']
    # Создание кубоида
    width, height, depth = dimensions['x'], dimensions['y'], dimensions['z']
    cuboid_3d = o3d.geometry.TriangleMesh.create_box(width, height, depth)
    cuboid_3d.paint_uniform_color([1, 0, 0]) # Цвети
    # Задание позиции
    x, y, z = position['x'], position['y'], position['z']
    cuboid_3d.translate([x, y, z])
    # Ротация
    alfa, beta, gamma = rotation['x'], rotation['y'], rotation['z']
    rotate = cuboid_3d.get_rotation_matrix_from_xyz((alfa, beta, gamma))
    cuboid_3d.rotate(rotate)
    # Получение облака точек
    cuboid = cuboid_3d.sample_points_uniformly(number_of_points=2000)
    return cuboid

In [328]:
with open('D:/university/6semestr/AI-diploma-niias/stasik/src/cloud_2_0001.pcd.json') as f:
    data = json.load(f)
figures = data['figures']

In [329]:
for i in range(len(figures)):
    geometries.append(create_cuboid(figures[i]['geometry']))


In [330]:
# downpcd = cloud.voxel_down_sample(voxel_size=0.05)
o3d.visualization.draw_geometries(geometries)

In [331]:
# points = np.asarray(cloud.points)
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(points[:,0], points[:,1], points[:,2])
# plt.show()